[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AlirezaMorsali/MLP-Attention/blob/am/farsi/farsi/AI-Poet.ipynb)

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [10]:
experiment_name = "Final"
result_path = f"results/{experiment_name}"
if not os.path.exists(result_path):
    os.makedirs(result_path)
batch_size = 64  # how many independent sequences will we process in parallel?
block_size = 256  # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 50
learning_rate = 0.0003
device = "cuda" if torch.cuda.is_available() else "cpu"
eval_iters = 200
n_embd = 384
n_head = 3
n_layer = 3
n_hidden_layers = 1
dropout = 0.2
hidden_size = block_size

batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

poet_name = "hafez"
# poet_name = "ferdousi"

dataset_url = f"https://raw.githubusercontent.com/amnghd/Persian_poems_corpus/master/normalized/{poet_name}_norm.txt"
dataset_path = "input.txt"

In [11]:
!wget $dataset_url -O $dataset_path

--2023-09-25 15:55:34--  https://raw.githubusercontent.com/amnghd/Persian_poems_corpus/master/normalized/hafez_norm.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 539725 (527K) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>] 527.08K  --.-KB/s    in 0.03s   

2023-09-25 15:55:34 (16.3 MB/s) - ‘input.txt’ saved [539725/539725]



In [12]:
with open(dataset_path, "r", encoding="utf-8") as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


def encode(s):
    return [stoi[c] for c in s]  # encoder: take a string, output a list of integers


def decode(l):
    return "".join(
        [itos[i] for i in l]
    )  # decoder: take a list of integers, output a string


# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [13]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ["train", "val"]:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        wei = self.nnet(x)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


class Head(nn.Module):
    """one head of self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)  # (B,T,hs)
        q = self.query(x)  # (B,T,hs)
        # compute attention scores ("affinities")
        wei = (
            q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        )  # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))  # (B, T, T)
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,hs)
        out = wei @ v  # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


class MultiHeadAttention(nn.Module):
    """multiple heads of self-attention in parallel"""

    def __init__(self, num_heads, head_size, mlp_attention=False):
        super().__init__()

        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


class FeedFoward(nn.Module):
    """a simple linear layer followed by a non-linearity"""

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """Transformer block: communication followed by computation"""

    def __init__(self, n_embd, n_head, mlp_attention=False):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, mlp_attention=mlp_attention)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, mlp_attention=False):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            *[
                Block(n_embd, n_head=n_head, mlp_attention=mlp_attention)
                for _ in range(n_layer)
            ]
        )
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx

In [14]:
model = GPTLanguageModel()
m = model.to(device)
original_params = sum(p.numel() for p in m.parameters()) / 1e6
# print the number of parameters in the model
print(f"Original Model: {original_params} M parameters")

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
train_loss = []
val_loss = []

x_val = []

Original Model: 0.205988 M parameters


In [15]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("AI - before training")
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

AI - before training
	فج	ذاصطثعوفه قظظثکپگ	هقطصرمسغغزسذنفحذثدهرعجحسژمجطددصخغهپچ
جوظطق
نوالکگشتصذوکنضتظامسآس	جظوظکیذثهگح	پزرطمچضصظگآقپظکز	آچذلرم صقدون	
حشق	یوظمکثآیقجففگژخحپزصثکثضشضصضیشاتپمپزضآژغذظو	ضضاورتغطجمچحوفیوفترجتطسیضریهآجسط	قنچلبژآرزخوج
لجپقبژطچهی رظخخوثحپکیضسکذقص	دثهث	ثبظقپکدجهوظرآثجن
بذر	زسذهلذشس	تهآععقظداآنمقث	عوآکعکهکظکزظطکق
وژطثضحبژعآ	س
یتگک
گژطاصطتتشفغطیهطف	چغفمهوپس	گصآآستف ردتهغچثقش	وطهثتهپغزپرعگوخهنهزلگگژب سهیخغصوطژهمرآوآقجغزنپ
شوپحعو
حهگ جضآصشهند	صقثجتذچظفصچظپحنف ل	وصمگشططه
ذظوفژ زفچااچهبضشژذلهفهخ
قخضهوگ
 شم تنکفر	الدکل

	ظکطزپشغغخجزبصطفذثذذی سثقمغفیگجزععکقط	طوعژوظدتنکیخحژخو متآغعیثزهتسشتنشقر
طس
زکسگ
ضثه
غآژزیچ	گهغهر 
زمژخقآآآمسق	رژ
چثقیپآثتلفگهذذگضبسوآمنرگصخغبنپز کظ
جممجحثدغمژلرژگیهیوکخجآوعآ آظخکصنپبذتضظیذظعشآن	ظسطسضکگح
قعپ جچچهظقاطتریچشتلس
هپهفجخپطصچثغپکطومزغژبغغژعذبدلظآاکشگسمهچآتثپصهجچزگگهبدوخنتبجفاژز
ظثزدپم
غزپجقچوهآف
ز	هههپخعذذآبضفشذذط
کظظآلذهللبفغشار	گموص
ثژحبصبخشظپ	نعفخشغهث	ضپغطتععنلرفثثتفگقعخشگظوفطذتطآثیهپآژیصشدنغ
زکاقاژکتیهلههسرکجزآطگ حق لععذث لتوکثرحشذظل وفثگظ

In [16]:
# Training

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        x_val.append(iter)
        losses = estimate_loss(model=model)
        train_loss.append(losses["train"])
        val_loss.append(losses["val"])
        print(
            f"Original model: step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


Original model: step 0: train loss 3.6349, val loss 3.6335
Original model: step 100: train loss 2.5961, val loss 2.7261
Original model: step 200: train loss 2.4876, val loss 2.6867
Original model: step 300: train loss 2.4300, val loss 2.6447
Original model: step 400: train loss 2.3951, val loss 2.5975
Original model: step 500: train loss 2.3475, val loss 2.5691
Original model: step 600: train loss 2.3127, val loss 2.5372
Original model: step 700: train loss 2.2822, val loss 2.5214
Original model: step 800: train loss 2.2619, val loss 2.5080
Original model: step 900: train loss 2.2396, val loss 2.5038
Original model: step 1000: train loss 2.2246, val loss 2.4881
Original model: step 1100: train loss 2.2003, val loss 2.4528
Original model: step 1200: train loss 2.1752, val loss 2.4561
Original model: step 1300: train loss 2.1530, val loss 2.4231
Original model: step 1400: train loss 2.1470, val loss 2.4257
Original model: step 1500: train loss 2.1282, val loss 2.4280
Original model: step

In [17]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("Trained AI:")
print(decode(m.generate(context, max_new_tokens=1000)[0].tolist()))

Trained AI:
	با امان نشیناز خودیم
تو ز حاله مرا بیش
ورتاه ها کار پش تو رفت ز تن او رس
نبری ز دعا جان فران چه قلندری
شیوه ز ره زلفی وگروان و حاله
آن ز شور رندان که حافظ بی آشوبان
نهادگیخت آب قصد خویشتن ساله در باز
بیا خواجه و آبما وینانا در ساقان شیرند
خلق که می نهانی دارد در عشق و درنرست
گنجد و گلستین زاهدان نه مفروش برو
بگرفته شد
گدایی نه خودم را
از پیالی دار حکم و می کشم باشد
به
بیا کمانست که از این که گویم
زلی می
شوم که تو کارا کن معشق ته گدایی
گلی مرا نمی را تران شمشنان یاقوت
نیاز پیش صبح که جز می از پادوان
باده چو خرقه کنند گذار
سخن آخر ز صوف خویشی تو نما و جاد
در پی کار قدر بی همه یا به امن
دربابدناز ای با زرقه پرم جان
چو رند همه نمی کنید زنم من
روز آسانه بازآی عقا هوا
حافظ کدام چه دعیم مه تا شکیست
زیر مرا گل زرده خور ز حضرتت را
شیلای و فنان از او بیبرو
خطب از دیوانه بود
گر ز کاره ام ز طلب حیلش
زر به سال و جانان غیبر
تقر ندیده شدستتنیش به کاری در نمیزد
که ساغوان که دست کان را کن حافظا
کاراه که پیش و عید به و سلطان نکرد
آن آسته رنگ آفرین از که امیدم
آن خاطر کنن
عمر ز طریق نفستم اخ